In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'forest_fires',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
# Keep numeric columns
X = []
for s in a[1].keys():
    if a[1][s].dtype != tf.string:
        X.append(a[1][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[0].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[0].numpy()[test_idx]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = np.inf

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = lgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['rmse'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'objective': hp.choice('objective',['regression']),
    'metric':hp.choice('metric',['rmse']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.7422799940439515, 'feature_fraction': 0.9213873364392791, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.0021431564062737904, 'metric': 'rmse', 'min_data_in_leaf': 85.0, 'min_sum_hessian_in_leaf': 1.1465123145471277, 'num_leaves': 2.0, 'objective': 'regression'}
[1]	valid_0's rmse: 24.1715                            
[2]	valid_0's rmse: 24.1732                            
[3]	valid_0's rmse: 24.1749                            
[4]	valid_0's rmse: 24.1766                            
[5]	valid_0's rmse: 24.1783                            
[6]	valid_0's rmse: 24.179                             
[7]	valid_0's rmse: 24.1808                            
[8]	valid_0's rmse: 24.1815                            
[9]	valid_0's rmse: 24.1833                            
[10]	valid_0's rmse: 24.1841                           
[11]	valid_0's rmse: 24.1859                           
[12]	valid_0's rmse: 24.1867                           
[13]	valid_0's rmse: 24.1887          

[24]	valid_0's rmse: 24.5777                                                     
[25]	valid_0's rmse: 24.6191                                                     
[26]	valid_0's rmse: 24.6546                                                     
[27]	valid_0's rmse: 24.6842                                                     
[28]	valid_0's rmse: 24.7071                                                     
[29]	valid_0's rmse: 24.754                                                      
[30]	valid_0's rmse: 24.8144                                                     
[31]	valid_0's rmse: 24.796                                                      
[32]	valid_0's rmse: 24.8193                                                     
[33]	valid_0's rmse: 24.8465                                                     
[34]	valid_0's rmse: 24.829                                                      
[35]	valid_0's rmse: 24.8158                                                     
[36]	valid_0's r

[19]	valid_0's rmse: 24.1967                                                    
[20]	valid_0's rmse: 24.1939                                                    
[21]	valid_0's rmse: 24.1965                                                    
[22]	valid_0's rmse: 24.1992                                                    
[23]	valid_0's rmse: 24.2019                                                    
[24]	valid_0's rmse: 24.2047                                                    
[25]	valid_0's rmse: 24.2074                                                    
[26]	valid_0's rmse: 24.2102                                                    
[27]	valid_0's rmse: 24.213                                                     
[28]	valid_0's rmse: 24.215                                                     
[29]	valid_0's rmse: 24.2166                                                    
[30]	valid_0's rmse: 24.2194                                                    
[31]	valid_0's rmse: 24.2212

[16]	valid_0's rmse: 24.7912                                                    
[17]	valid_0's rmse: 24.845                                                     
[18]	valid_0's rmse: 24.8238                                                    
[19]	valid_0's rmse: 24.8805                                                    
[20]	valid_0's rmse: 24.8831                                                    
[21]	valid_0's rmse: 24.9434                                                    
[22]	valid_0's rmse: 25.0072                                                    
[23]	valid_0's rmse: 25.018                                                     
[24]	valid_0's rmse: 24.9964                                                    
[25]	valid_0's rmse: 24.9607                                                    
[26]	valid_0's rmse: 25.0228                                                    
[27]	valid_0's rmse: 25.0902                                                    
[28]	valid_0's rmse: 25.063 

[13]	valid_0's rmse: 24.678                                                     
[14]	valid_0's rmse: 24.9678                                                    
[15]	valid_0's rmse: 25.0747                                                    
[16]	valid_0's rmse: 25.1441                                                    
[17]	valid_0's rmse: 25.3089                                                    
[18]	valid_0's rmse: 25.4105                                                    
[19]	valid_0's rmse: 25.5576                                                    
[20]	valid_0's rmse: 25.5331                                                    
[21]	valid_0's rmse: 25.6018                                                    
[22]	valid_0's rmse: 25.7042                                                    
[23]	valid_0's rmse: 25.83                                                      
[24]	valid_0's rmse: 25.9809                                                    
[25]	valid_0's rmse: 26.1825

...

[72]	valid_0's rmse: 23.6166                                                      
[73]	valid_0's rmse: 23.6242                                                      
[74]	valid_0's rmse: 23.6269                                                      
[75]	valid_0's rmse: 23.6308                                                      
[76]	valid_0's rmse: 23.6073                                                      
[77]	valid_0's rmse: 23.6241                                                      
[78]	valid_0's rmse: 23.6388                                                      
[79]	valid_0's rmse: 23.646                                                       
[80]	valid_0's rmse: 23.6529                                                      
[81]	valid_0's rmse: 23.6578                                                      
[82]	valid_0's rmse: 23.6321                                                      
[83]	valid_0's rmse: 23.6311                                                      
[84]

[66]	valid_0's rmse: 24.1883                                                      
[67]	valid_0's rmse: 24.1904                                                      
[68]	valid_0's rmse: 24.1922                                                      
[69]	valid_0's rmse: 24.1937                                                      
[70]	valid_0's rmse: 24.194                                                       
[71]	valid_0's rmse: 24.1956                                                      
[72]	valid_0's rmse: 24.1947                                                      
[73]	valid_0's rmse: 24.1961                                                      
[74]	valid_0's rmse: 24.1952                                                      
[75]	valid_0's rmse: 24.1966                                                      
[76]	valid_0's rmse: 24.1959                                                      
[77]	valid_0's rmse: 24.1955                                                      
[78]

[60]	valid_0's rmse: 39.3906                                                      
[61]	valid_0's rmse: 39.6816                                                      
[62]	valid_0's rmse: 40.2713                                                      
[63]	valid_0's rmse: 40.512                                                       
[64]	valid_0's rmse: 40.7868                                                      
[65]	valid_0's rmse: 40.8202                                                      
[66]	valid_0's rmse: 41.1772                                                      
[67]	valid_0's rmse: 41.4306                                                      
[68]	valid_0's rmse: 41.4571                                                      
[69]	valid_0's rmse: 41.4938                                                      
[70]	valid_0's rmse: 41.7263                                                      
[71]	valid_0's rmse: 41.7439                                                      
[72]

[54]	valid_0's rmse: 24.2048                                                      
[55]	valid_0's rmse: 24.2083                                                      
[56]	valid_0's rmse: 24.2109                                                      
[57]	valid_0's rmse: 24.2139                                                      
[58]	valid_0's rmse: 24.2111                                                      
[59]	valid_0's rmse: 24.213                                                       
[60]	valid_0's rmse: 24.216                                                       
[61]	valid_0's rmse: 24.2188                                                      
[62]	valid_0's rmse: 24.216                                                       
[63]	valid_0's rmse: 24.2132                                                      
[64]	valid_0's rmse: 24.2105                                                      
[65]	valid_0's rmse: 24.2136                                                      
[66]

[48]	valid_0's rmse: 24.1888                                                      
[49]	valid_0's rmse: 24.1881                                                      
[50]	valid_0's rmse: 24.1913                                                      
[51]	valid_0's rmse: 24.1947                                                      
[52]	valid_0's rmse: 24.1939                                                      
[53]	valid_0's rmse: 24.1963                                                      
[54]	valid_0's rmse: 24.1952                                                      
[55]	valid_0's rmse: 24.198                                                       
[56]	valid_0's rmse: 24.2004                                                      
[57]	valid_0's rmse: 24.2034                                                      
[58]	valid_0's rmse: 24.2006                                                      
[59]	valid_0's rmse: 24.1984                                                      
[60]

In [4]:
print(best_val)
print(space_eval(spc, best))

23.493418985793742
{'bagging_fraction': 0.5088579314076509, 'feature_fraction': 0.5259726644212983, 'lambda_l1': 2.369294834188504e-07, 'lambda_l2': 0, 'learning_rate': 0.0033233529278171624, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 0.0005659274065460817, 'num_leaves': 291.0, 'objective': 'regression'}


In [ ]:
# Trial 1
#63.30630673002118
#{'bagging_fraction': 0.8562372517785388, 'feature_fraction': 0.5978632262605562, 'lambda_l1': 1.671529213829888e-05, 'lambda_l2': 0, 'learning_rate': 0.8654043007137826, 'metric': 'rmse', 'min_data_in_leaf': 19.0, 'min_sum_hessian_in_leaf': 0.00020689178853106868, 'num_leaves': 314.0, 'objective': 'regression'}

# Trial 2
#26.924289634554277
#{'bagging_fraction': 0.6944367025871995, 'feature_fraction': 0.7048574512000494, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.0027504662370871126, 'metric': 'rmse', 'min_data_in_leaf': 105.0, 'min_sum_hessian_in_leaf': 0.4958014070842451, 'num_leaves': 60.0, 'objective': 'regression'}

# Trial 3
#31.653419316397716
#{'bagging_fraction': 0.5862110937921585, 'feature_fraction': 0.9774280027807499, 'lambda_l1': 0, 'lambda_l2': 0.2024656829317716, 'learning_rate': 0.06499097041822029, 'metric': 'rmse', 'min_data_in_leaf': 8.0, 'min_sum_hessian_in_leaf': 1.2108766975436134, 'num_leaves': 19.0, 'objective': 'regression'}

# Trial 4
#31.719169301155414
#{'bagging_fraction': 0.6679767139736139, 'feature_fraction': 0.5553541370325076, 'lambda_l1': 0, 'lambda_l2': 0, 'learning_rate': 0.003442920523588154, 'metric': 'rmse', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.23022923591022698, 'num_leaves': 137.0, 'objective': 'regression'}


# Trial 5
#23.493418985793742
#{'bagging_fraction': 0.5088579314076509, 'feature_fraction': 0.5259726644212983, 'lambda_l1': 2.369294834188504e-07, 'lambda_l2': 0, 'learning_rate': 0.0033233529278171624, 'metric': 'rmse', 'min_data_in_leaf': 2.0, 'min_sum_hessian_in_leaf': 0.0005659274065460817, 'num_leaves': 291.0, 'objective': 'regression'}